In [6]:
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

In [7]:
# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # Bypass detection
chrome_options.add_argument("--window-size=1,1")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36")  # User-agent string

# Disable automation controls so that websites don't detect Selenium
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

# Crawl links

### Crawl links mua bán đất from nhatot.com

In [4]:
# Initialize the array to store the links
arr = []
start_page = 1
end_page = 1

# Retry logic
MAX_RETRIES = 3

for page in range(start_page, end_page + 1):
    for attempt in range(MAX_RETRIES):
        try:
            # Initialize WebDriver for each page
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            url = f"https://www.nhatot.com/mua-ban-dat-tp-ho-chi-minh?page={page}"
            driver.get(url)

            # Use explicit wait for better control
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, 'AdItem_adItem__gDDQT')))

            # Parse the page source with BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Find all the links with the specified class
            a_tags = soup.find_all("a", class_='AdItem_adItem__gDDQT')

            # Append the full link to the array
            for a in a_tags:
                full_link = "https://www.nhatot.com" + a['href']
                arr.append(full_link)

            driver.quit()
            break  # Break if the request was successful
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            driver.quit()
            if attempt + 1 == MAX_RETRIES:
                print("Max retries reached. Skipping this page.")
            time.sleep(5)  # Wait before retrying

# Create a DataFrame from the array
df = pd.DataFrame(arr, columns=["Links"])

# Export the DataFrame to a CSV file
df.to_csv('muaBanDatLinks.csv', index=True)

print("done")

done


# Lấy chi tiết data

### Lấy chi tiết data mua bán đất

In [8]:
from tenacity import retry, wait_fixed, stop_after_attempt

# Function to extract data from a single page
@retry(wait=wait_fixed(2), stop=stop_after_attempt(3))  # Wait 2 seconds between retries, retry 3 times
def extract_data(url):
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

    try:
        driver.get(url)

        # Wait until price element is available or timeout after 10 seconds
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'cd9gm5n')))

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        try:
            title = soup.find("title").text
        except AttributeError:
            title = None

        try:
            price = soup.find("b", class_='pyhk1dv').text
        except AttributeError:
            price = None

        try:
            address = soup.find("span", class_="flex-1").text
        except (IndexError, AttributeError):
            address = None

        try:
            price_m2 = soup.find("span", itemprop="price_m2").text
        except AttributeError:
            price_m2 = None

        try:
            direction = soup.find("span", itemprop="direction").text
        except AttributeError:
            direction = None
            
        try:
            property_legal_document = soup.find("span", itemprop="property_legal_document").text
        except AttributeError:
            property_legal_document = None

        try:
            land_type = soup.find("span", itemprop="land_type").text
        except AttributeError:
            land_type = None

        try:
            pty_characteristics = soup.find("span", itemprop="pty_characteristics").text
        except AttributeError:
            pty_characteristics = None

        try:
            size = soup.find("span", itemprop="size").text
        except AttributeError:
            size = None
        
        try:
            width = soup.find("span", itemprop="width").text
        except AttributeError:
            width = None

        try:
            length = soup.find("span", itemprop="length").text
        except AttributeError:
            length = None
        
        driver.quit()

        return {
            "title": title,
            "price": price,
            "address": address,
            "price_m2": price_m2,
            "direction": direction,
            "property_legal_document": property_legal_document,
            "land_type": land_type,
            "pty_characteristics": pty_characteristics,
            "size": size,
            "width": width,
            "length": length,
        }
        
    except Exception as e:
        print(f"Error processing URL {url}: {e}")
        driver.quit()
        return None

# Function to save data to CSV
def save_to_csv(data_list, filename='muaBanDat.csv'):
    if not data_list:
        print("No data to save.")
        return
    keys = data_list[0].keys()  # Use the keys from the first dictionary as headers
    with open(filename, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()
        dict_writer.writerows(data_list)

# Main function to process all URLs and save results
def main():
    # Replace this with your actual DataFrame or list of URLs
    frame = pd.read_csv('muaBanDatLinks.csv')  # Assuming you already have the 'Links' data in CSV
    all_data = []
    i = 0
    for url in frame['Links']:
        data = extract_data(url)
        i+=1
        if data:
            all_data.append(data)
        print(f"Processed URL {i}: {url}")

    save_to_csv(all_data)

if __name__ == "__main__":
    main()

Processed URL 1: https://www.nhatot.com/mua-ban-dat-thanh-pho-thu-duc-tp-ho-chi-minh/119642177.htm#px=SR-stickyad-[PO-1][PL-top]
Processed URL 2: https://www.nhatot.com/mua-ban-dat-huyen-cu-chi-tp-ho-chi-minh/118197275.htm#px=SR-stickyad-[PO-2][PL-top]
Processed URL 3: https://www.nhatot.com/mua-ban-dat-quan-binh-tan-tp-ho-chi-minh/119642896.htm#px=SR-stickyad-[PO-3][PL-top]


In [ ]:
frame1 = pd.read_csv('muaBanDat.csv')
frame1.head()